In [ ]:
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install torch torchaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import matplotlib.pyplot as plt
df = pd.read_csv("/content/lyrics.csv", engine = 'python', encoding='utf-8', error_bad_lines=False)
df

<ipython-input-34-3b60a70b98d3>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("/content/lyrics.csv", engine = 'python', encoding='utf-8', error_bad_lines=False)


,Unnamed: 0,link,artist,song_name,lyrics
0,0,../lyrics/backstreetboys/climbingthewalls.html,Backstreet Boys Lyrics,Climbing The Walls,"\n\n[Brian:]\nClose your eyes, make a wish\nTh..."
1,1,../lyrics/westlife/howtobreakaheart.html,Westlife Lyrics,How To Break A Heart,\n\r\nSince you're not worth my love\nI haven'...
2,2,../lyrics/deanmartin/iwill.html,Dean Martin Lyrics,I Will,\n\r\nI don't want to be the one to say I'm go...
3,3,../lyrics/deanmartin/tellyourmotherhello.html,Dean Martin Lyrics,Tell Your Mother Hello,\n\r\nSure I loved the dear money that shines ...
4,4,../lyrics/deanmartin/behonestwithme.html,Dean Martin Lyrics,Be Honest With Me,\n\r\nBe honest with me dear whatever you do\n...
...,...,...,...,...,...
25737,25737,../lyrics/jasonmraz/whatmamasay.html,Jason Mraz Lyrics,What Mama Say,\n\r\n(La la da da da da...)\n\nWhatever mama ...
25738,25738,https://www.azlyrics.com/lyrics/gesaffelstein/...,Gesaffelstein Lyrics,Blast Off,\n\r\nYou've been thinking about this all nigh...
25739,25739,../lyrics/jasonmraz/aworldwithyou.html,Jason Mraz Lyrics,A World With You,\n\r\nLet's hit the road and throw out the map...
25740,25740,../lyrics/jasonmraz/loveisreal.html,Jason Mraz Lyrics,Love Is Real,\n\r\nLove is real it is not just in novels or...


In [ ]:
df = df.drop(df.columns[0], axis=1)

RECORD AUDIO

In [ ]:
!pip install ffmpeg-python

In [ ]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
import scipy

In [ ]:
audio, sr = get_audio()
scipy.io.wavfile.write('recording.wav', sr, audio)

In [ ]:
!pip install speechrecognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 32.1 MB/s eta 0:00:00


In [ ]:
!pip install gtts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from gtts import gTTS
from IPython.display import Audio

In [ ]:
import speech_recognition as sr
r = sr.Recognizer()
suara = sr.AudioFile('/content/recording.wav')
with suara as source:
  audio = r.record(source)
Sample_lyrics = r.recognize_google(audio, language='en-EN')
lirik = df[['artist','lyrics']].values
df['match_score'] = df['lyrics'].apply(lambda x: fuzz.partial_ratio(x, Sample_lyrics))
sorted_df = df.sort_values('match_score', ascending=False)
sorted_df[0:3]
tts = gTTS(f'The most simmilar song to your search is :{sorted_df.song_name[0:1].values[0]} by {sorted_df.artist[0:1].values[0]} with a match score of {sorted_df.match_score[0:1].values[0]}')
tts.save('1.wav')
sound_file = '1.wav'
Audio(sound_file, autoplay=True)

In [ ]:
sorted_df[0:3]

,Unnamed: 0,link,artist,song_name,lyrics,match_score
6499,6499,../lyrics/charlieputh/seeyouagainpianodemovers...,Charlie Puth Lyrics,See You Again (Piano Demo Version),"\n\r\nIt's been a long day without you, my fri...",98
6467,6467,https://www.azlyrics.com/lyrics/wizkhalifa/see...,Wiz Khalifa Lyrics,See You Again,\n\n[Charlie Puth:]\nIt's been a long day with...,98
22927,22927,../lyrics/macmiller/right.html,Mac Miller Lyrics,Right,"\n\r\nYeah, it's been a while without your fac...",67


In [ ]:
Sample_lyrics

"it's been a long day without you my friend"